In [1]:
import os
import requests
import json
import pandas as pd


In [2]:
# Get image data from csv

image_data = pd.read_csv('demo_data/image_data.csv')


In [3]:
# This is how to access the data

image_data.iloc[0]['file_name']
image_data.iloc[0]['im_id']


'IM_ZCLKsKOa8J3KMQC'

In [4]:
# WhoAmI

# Set up & check
api_token = os.environ.get('api_token')
data_center_id = os.environ.get('data_center_id')

whoami_url = f"https://{data_center_id}.qualtrics.com/API/v3/whoami"

headers = {
    "Content-Type": "application/json",
    "X-API-TOKEN": api_token
}

response = requests.request("GET", whoami_url, headers=headers)

pretty_response = json.dumps(response.json(), indent=4)

print(pretty_response)


{"result":{"brandId":"uniluzern","userId":"UR_07fDIefOcsc7794","userName":"maurice.flechtner@stud.unilu.ch#uniluzern","accountType":"UT_3dBUKOs5wAT2mLW","firstName":"Maurice","lastName":"Flechtner","email":"maurice.flechtner@stud.unilu.ch","datacenter":"awsireland"},"meta":{"requestId":"441496ab-9581-4961-aa9e-1d03578a6ca0","httpStatus":"200 - OK"}}


In [ ]:
# Get the Library id

url = "https://{0}.qualtrics.com/API/v3/libraries/".format(
    data_center_id)

headers = {
    "Content-Type": "application/json",
    "X-API-TOKEN": api_token
}

response = requests.request("GET", url, headers=headers)

library_id = json.loads(response.text)['result']['elements'][0]['libraryId']

pretty_response = json.dumps(response.json(), indent=4)

print(pretty_response)


In [26]:
# Create a new survey

survey_name = "Real API Survey Test"

baseUrl = "https://{0}.qualtrics.com/API/v3/survey-definitions".format(
    data_center_id)
headers = {
    "x-api-token": api_token,
    "content-type": "application/json",
    "Accept": "application/json"
}

data = {"SurveyName": survey_name,"Language": "EN","ProjectCategory": "CORE"}

response = requests.post(baseUrl, json=data, headers=headers)

survey_id = json.loads(response.text)['result']['SurveyID']

pretty_response = json.dumps(response.json(), indent=4)

print(pretty_response)


{
    "meta": {
        "httpStatus": "200 - OK",
        "requestId": "f4c30dd5-a890-41ca-8682-b16ee8af838d"
    },
    "result": {
        "SurveyID": "SV_dipUu69Szmv8Kou",
        "DefaultBlockID": "BL_eWJmq09gN1idVDE"
    }
}


In [27]:
# Post a question for each image

baseUrl = "https://{0}.qualtrics.com/API/v3/survey-definitions/{1}/questions".format(
        data_center_id, survey_id)

headers = {
'accept': "application/json",
'content-type': "application/json",
"x-api-token": api_token,
}

for i in range(len(image_data)):

    # Post an Image

    image_def = {
            "QuestionText": "Please have a look at this image:<br>",
                    "QuestionJS": "Qualtrics.SurveyEngine.addOnload(function() {\n var inputId = 'QR~' + this.questionId;\n var picker = new Pikaday(\n    {\n        field: document.getElementById(inputId),\n        firstDay: 1,\n        minDate: new Date(),\n        maxDate: new Date(2020, 12, 31),\n        yearRange: [2000,2020]\n\t});\n});\n\nQualtrics.SurveyEngine.addOnReady(function()\n{\n\t/*Place your JavaScript here to run when the page is fully displayed*/\n\n});\n\nQualtrics.SurveyEngine.addOnUnload(function()\n{\n\t/*Place your JavaScript here to run when the page is unloaded*/\n\n});",
                    "DefaultChoices": False,
                    "DataExportTag": f"IM{i}",
                    "QuestionID": f"IMID{i}",
                    "QuestionType": "DB",
                    "Selector": "GRB",
                    "DataVisibility": {
                        "Private": False,
                        "Hidden": False
                    },
                    "Configuration": {
                        "QuestionDescriptionOption": "UseText"
                    },
                    "QuestionDescription": "Please have a look at this image:",
                    "ChoiceOrder": [],
                    "Validation": {
                        "Settings": {
                            "Type": "None"
                        }
                    },
                    "GradingData": [],
                    "Language": [],
                    "NextChoiceId": 4,
                    "NextAnswerId": 1,
                    "SubSelector": "WTXB",
                    "Graphics": f"{image_data.iloc[i]['im_id']}",
                    "GraphicsDescription": f"image {image_data.iloc[i]['file_name']}",
                    "QuestionText_Unsafe": "Please have a look at this image:<br>"
                }

    response_0 = requests.post(baseUrl, json=image_def, headers=headers)

    # Post a multiple choice question

    question_def = {
         "QuestionText": "Rate the authenticity of this image:",
                "DefaultChoices": False,
                "DataExportTag": f"Q{i}",
                "QuestionType": "Slider",
                "Selector": "HSLIDER",
                "Configuration": {
                    "QuestionDescriptionOption": "UseText",
                    "CSSliderMin": 0,
                    "CSSliderMax": 100,
                    "GridLines": 10,
                    "SnapToGrid": False,
                    "NumDecimals": "0",
                    "ShowValue": True,
                    "CustomStart": False,
                    "NotApplicable": False,
                    "MobileFirst": True
                },
                "QuestionDescription": "Rate the authenticity of this image:",
                "Choices": {
                    "1": {
                        "Display": "Authenticity"
                    },
                    "2": {
                        "Display": "Product Placement"
                    },
                    "3": {
                        "Display": "Originality"
                    }
                },
                "ChoiceOrder": [
                    "1",
                    "2",
                    "3"
                ],
                "Validation": {
                    "Settings": {
                        "ForceResponse": "OFF",
                        "Type": "None"
                    }
                },
                "GradingData": [],
                "Language": [],
                "NextChoiceId": 4,
                "NextAnswerId": 1,
                "Labels": [],
                "QuestionID": f"QID{i}",
                "QuestionText_Unsafe": "Rate the authenticity of this image:"
            }
    response_1 = requests.post(baseUrl, json=question_def, headers=headers)

pretty_response = json.dumps(response_0.json(), indent=4)

print(pretty_response)


{
    "meta": {
        "httpStatus": "200 - OK",
        "requestId": "cf9b5d6d-7597-411d-9534-e2dcff41191d"
    },
    "result": {
        "QuestionID": "QID48"
    }
}


In [25]:
pretty_response = json.dumps(response_1.json(), indent=4)

print(pretty_response)


{
    "meta": {
        "requestId": "9e49a851-b2c3-48f4-a669-4f2767af10b4",
        "httpStatus": "403 - Forbidden",
        "error": {
            "errorCode": "QMST_2.1",
            "errorMessage": "The operation is forbidden."
        }
    }
}
